# Data Warehouse Iqvia QA - Member Enrollment Yearly

Performing QA on tables in dw_staging before moving them to data_warehouse schema

## Initialization

Just loading packages that will be used and initializing connection to GP DB.

In [1]:
import pandas as pd
import sys
import psycopg2
from tqdm.notebook import tqdm
sys.path.append('H:/uth_helpers')
from db_utils import get_dsn

In [2]:
connection = psycopg2.connect(get_dsn())
connection.autocommit = True

## Member Counts and Member Months

This table is an aggregate of the member_enrollment_monthly table where enrollment data is aggregated to a yearly level.

Because of this, we cannot compare the row count of the raw tables with this table since enrollment tables from raw data sources are usually not on a yearly level. Instead, we can try to compare the Member Months (MM) and other columns to see if the aggregrate logic implemented is working as intended

In [3]:
query = '''drop table if exists qa_reporting.dw_iqva_mbr_enrl_yearly;
create table qa_reporting.dw_iqva_mbr_enrl_yearly
(
    data_source text,
    calendar_year int,
    table_src text,
    dw_row_count int,
    dw_uth_mbr_id_count int,
    dw_src_mbr_id_count int,
    src_mbr_count int,
    mbr_count_diff int,
    mbr_count_percentage float,
    dw_mm int,
    src_mm int,
    mm_diff int,
    mm_diff_percentage float,
    date_generated date
);
'''

with connection.cursor() as cursor:
    cursor.execute(query)

In [4]:
with connection.cursor() as cursor:
      query = '''
insert into qa_reporting.dw_iqva_mbr_enrl_yearly
(data_source, calendar_year, table_src, dw_row_count, dw_uth_mbr_id_count, dw_src_mbr_id_count, dw_mm,date_generated)
select data_source, 
        year, 
        table_id_src, 
        count(*),
        count(distinct uth_member_id),
        count(distinct member_id_src),
        sum(total_enrolled_months),
        now()::date
  from dw_staging.iqva_member_enrollment_yearly
 group by 1,2,3;
      '''

      cursor.execute(query)


In [5]:
with connection.cursor() as cursor:
      query = '''
update qa_reporting.dw_iqva_mbr_enrl_yearly a
set src_mbr_count = b.pat_count,
      mbr_count_diff = a.dw_src_mbr_id_count - b.pat_count,
      mbr_count_percentage = 100. * abs(a.dw_src_mbr_id_count - b.pat_count) / b.pat_count
from qa_reporting.iqvia_counts b
where data_source = 'iqva'
and a.calendar_year = b.year
and a.table_src ||'2' = b.table_name
;
      '''

      cursor.execute(query)

      query = '''
with enrl_months as (
      select distinct year, pat_id, month_id
      from iqvia.enroll2
),
enrl_my as (
      select year, count(*) as mm
      from enrl_months
      group by 1
)
update qa_reporting.dw_iqva_mbr_enrl_yearly a
set src_mm = b.mm,
    mm_diff = a.dw_mm - b.mm,
    mm_diff_percentage = abs(a.dw_mm - b.mm) / b.mm
from enrl_my b
where data_source = 'iqva'
and a.calendar_year = b.year
;
      '''

      cursor.execute(query)

In [6]:
df = pd.read_sql('''select * 
from qa_reporting.dw_iqva_mbr_enrl_yearly 
order by mm_diff ;''', con=connection)
df

C:\Users\iperez9\.conda\envs\psycog_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,data_source,calendar_year,table_src,dw_row_count,dw_uth_mbr_id_count,dw_src_mbr_id_count,src_mbr_count,mbr_count_diff,mbr_count_percentage,dw_mm,src_mm,mm_diff,mm_diff_percentage,date_generated
0,iqva,2021,enroll,10444895,10444895,10444895,10444895,0,0.0,102692788,102692788,0,0.0,2024-01-18
1,iqva,2006,enroll,30675914,30675914,30675914,30675914,0,0.0,284739058,284739058,0,0.0,2024-01-18
2,iqva,2013,enroll,25579373,25579373,25579373,25579373,0,0.0,247873630,247873630,0,0.0,2024-01-18
3,iqva,2012,enroll,28221335,28221335,28221335,28221335,0,0.0,269545623,269545623,0,0.0,2024-01-18
4,iqva,2011,enroll,31088358,31088358,31088358,31088358,0,0.0,304518538,304518538,0,0.0,2024-01-18
5,iqva,2014,enroll,28543784,28543784,28543784,28543784,0,0.0,258722696,258722696,0,0.0,2024-01-18
6,iqva,2016,enroll,27306135,27306135,27306135,27306135,0,0.0,264487431,264487431,0,0.0,2024-01-18
7,iqva,2022,enroll,10691149,10691149,10691149,10691149,0,0.0,99941204,99941204,0,0.0,2024-01-18
8,iqva,2017,enroll,23776369,23776369,23776369,23776369,0,0.0,192155065,192155065,0,0.0,2024-01-18
9,iqva,2019,enroll,13625173,13625173,13625173,13625173,0,0.0,128691131,128691131,0,0.0,2024-01-18


Here we look at the overall difference of member enrollment and member count. We start of by just adding the counts for each of the source tables. However, due to how the yearly table is built, it may be better to look at member counts and member months as the data source overall.

In [7]:
df.groupby('calendar_year')[['mm_diff', 'mm_diff_percentage', 'mbr_count_diff', 'mbr_count_percentage']].sum()

,mm_diff,mm_diff_percentage,mbr_count_diff,mbr_count_percentage
calendar_year,,,,
2006,0,0.0,0,0.0
2007,0,0.0,0,0.0
2008,0,0.0,0,0.0
2009,0,0.0,0,0.0
2010,0,0.0,0,0.0
2011,0,0.0,0,0.0
2012,0,0.0,0,0.0
2013,0,0.0,0,0.0
2014,0,0.0,0,0.0


## Gender Counts

Just like with the monthly enrollment table, we need to check the counts and values of the other columns. Here we take a look at the gender_cd column. Note that there are people for which we are not given their demographic data, meaning that we can ignore such rows

In [16]:
query = '''with iqva_gen_cd as (
    select distinct year, a.pat_id, der_sex
    from iqvia.enroll2 a
    join iqvia.enroll_synth b
    on a.pat_id = b.pat_id
),
iqva_gen as (
    select year, der_sex, count(*) gender_count
    from iqva_gen_cd
    group by 1,2
), dw_gen as (
    select year, gender_cd, count(*) gender_count
    from dw_staging.iqva_member_enrollment_yearly
    group by 1,2
)
select a.*, b.gender_count as src_gender_count, 
        a.gender_count - b.gender_count as gender_diff, 
        100. * abs(a.gender_count - b.gender_count) / b.gender_count as gender_diff_percent
from iqva_gen b
join dw_gen a
on a.year = b.year
and a.gender_cd = b.der_sex
order by year, gender_cd;
'''

pd.read_sql(query,  con=connection)

C:\Users\iperez9\.conda\envs\psycog_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,gender_cd,gender_count,src_gender_count,gender_diff,gender_diff_percent
0,2006,F,1154719,1154719,0,0.0
1,2006,M,1053423,1053423,0,0.0
2,2006,U,44,44,0,0.0
3,2007,F,1214513,1214513,0,0.0
4,2007,M,1110803,1110803,0,0.0
5,2007,U,44,44,0,0.0
6,2008,F,1271276,1271276,0,0.0
7,2008,M,1166103,1166103,0,0.0
8,2008,U,43,43,0,0.0
9,2009,F,1299655,1299655,0,0.0


## Plan Type

Check if plan types are properly mapped at a yearly level. If the plan type counts do not match between the member_enrollment_yearly table and the raw enrollment table, it most likely occurred when cleaning up the plan_type column. Ideally the count difference percent should be <= 1%.

In [21]:
query = '''with iqva_enroll as (
    select year, pat_id, plan_type, month_id
    from iqvia.enroll2 a
    left join reference_tables.ref_plan_type c
  	on c.data_source  = 'iqva' 
    and c.plan_type_src = a.prd_type
),
iqva_agg_enroll as (
    select year, pat_id, plan_type, max(month_id) my, count(*)
    from iqva_enroll
    group by 1,2,3
),
iqva_grp_enroll as (
    select *, row_number() over(partition by pat_id, year order by count desc, my desc) as my_grp
    from iqva_agg_enroll
),
iqva_plans as (
    select year, plan_type, count(*) plan_count
    from iqva_grp_enroll 
    where my_grp = 1
    group by 1,2
),
dw_plans as (
    select year, plan_type, count(*) plan_count
    from dw_staging.iqva_member_enrollment_yearly
    group by 1,2
)
select a.*, b.plan_count as src_plan_count, 
        a.plan_count - b.plan_count as plan_diff, 
        100. * abs(a.plan_count - b.plan_count) / b.plan_count as plan_diff_percent
from iqva_plans b
join dw_plans a
on a.year = b.year
and a.plan_type = b.plan_type
order by year;
'''

plan_type_counts_df = pd.read_sql(query,  con=connection)
plan_type_counts_df

C:\Users\iperez9\.conda\envs\psycog_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
0,2006,UNK,230742,230742,0,0.000000
1,2006,CDHP,96372,96372,0,0.000000
2,2006,POS,2450542,2450526,16,0.000653
3,2006,FFS,1426928,1426928,0,0.000000
4,2006,HMO,8494010,8493640,370,0.004356
...,...,...,...,...,...,...
101,2022,HMO,2985728,2985728,0,0.000000
102,2023,PPO,4196810,4196810,0,0.000000
103,2023,UNK,196659,196659,0,0.000000
104,2023,CDHP,1195719,1195719,0,0.000000


In [22]:
plan_type_counts_df.groupby('year')['plan_diff_percent'].min(), plan_type_counts_df.groupby('year')['plan_diff_percent'].max()

(year
 2006    0.0
 2007    0.0
 2008    0.0
 2009    0.0
 2010    0.0
 2011    0.0
 2012    0.0
 2013    0.0
 2014    0.0
 2015    0.0
 2016    0.0
 2017    0.0
 2018    0.0
 2019    0.0
 2020    0.0
 2021    0.0
 2022    0.0
 2023    0.0
 Name: plan_diff_percent, dtype: float64,
 year
 2006    0.004356
 2007    0.025240
 2008    0.009730
 2009    0.004635
 2010    0.005091
 2011    0.006395
 2012    0.008457
 2013    0.020564
 2014    0.016726
 2015    0.097561
 2016    0.329489
 2017    0.007529
 2018    0.000000
 2019    0.000000
 2020    0.000000
 2021    0.000000
 2022    0.000000
 2023    0.000000
 Name: plan_diff_percent, dtype: float64)

In [23]:
plan_type_counts_df.sort_values('plan_diff_percent')

,year,plan_type,plan_count,src_plan_count,plan_diff,plan_diff_percent
0,2006,UNK,230742,230742,0,0.000000
80,2018,HMO,3235747,3235747,0,0.000000
79,2018,FFS,219,219,0,0.000000
78,2018,CDHP,1230631,1230631,0,0.000000
77,2018,POS,555952,555952,0,0.000000
...,...,...,...,...,...,...
7,2007,HMO,8472771,8470633,2138,0.025240
66,2016,HMO,10412895,10410198,2697,0.025907
67,2016,POS,963062,962757,305,0.031680
61,2015,FFS,1026,1025,1,0.097561


In [24]:
plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()
plan_types_agg_df['percent_diff'] = 100. * plan_types_agg_df['plan_diff'] / plan_types_agg_df['src_plan_count'] 
plan_types_agg_df

C:\Users\iperez9\AppData\Local\Temp\80\ipykernel_30352\1871058508.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  plan_types_agg_df = plan_type_counts_df.groupby('year')['plan_count', 'src_plan_count', 'plan_diff'].sum()


,plan_count,src_plan_count,plan_diff,percent_diff
year,,,,
2006,30673576,30673174,402,0.001311
2007,34805575,34803305,2270,0.006522
2008,37224619,37223637,982,0.002638
2009,35200915,35200456,459,0.001304
2010,31377523,31377138,385,0.001227
2011,31072095,31071435,660,0.002124
2012,28202333,28201628,705,0.002500
2013,25556325,25554490,1835,0.007181
2014,28508226,28506519,1707,0.005988


## State

In [25]:
query = '''with iqva_enroll as (
    select distinct year, a.pat_id, coalesce(pat_state, 'XX') pat_state
    from iqvia.enroll2 a
    left join iqvia.enroll_synth b
    on a.pat_id = b.pat_id
),
iqva_states as (
    select year, pat_state, count(*) state_count
    from iqva_enroll m
    group by 1,2
), 
dw_plans as (
    select year, case when state is null then 'XX' else state end as state, count(*) state_count
    from dw_staging.iqva_member_enrollment_yearly
    group by 1,2
)
select a.*, b.state_count as src_state_count, 
        a.state_count - b.state_count as plan_diff, 
        100. * abs(a.state_count - b.state_count) / b.state_count as plan_diff_percent
from iqva_states b
full join dw_plans a
on a.year = b.year
and a.state = b.pat_state
order by year;
'''

df = pd.read_sql(query,  con=connection)
df

C:\Users\iperez9\.conda\envs\psycog_env\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
0,2006,KS,1109,1109,0,0.0
1,2006,VA,292377,292377,0,0.0
2,2006,PA,5215,5215,0,0.0
3,2006,FL,22227,22227,0,0.0
4,2006,AK,177,177,0,0.0
...,...,...,...,...,...,...
931,2023,RI,4828,4828,0,0.0
932,2023,NC,63280,63280,0,0.0
933,2023,KS,10545,10545,0,0.0
934,2023,TX,120780,120780,0,0.0


We see that there is a large difference in state values assigned using DW logic compared to raw data. Ideally the difference should show that there are more values in the raw data compared to the values in DW due, especially for state values of 'XX'. When this happens, it means that the member now has a valid state value which can be used to group these member in future projects.

In [26]:
df.sort_values('plan_diff_percent', ascending=False)

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
0,2006,KS,1109,1109,0,0.0
628,2018,KS,7122,7122,0,0.0
616,2017,MT,2874,2874,0,0.0
617,2017,MI,22450,22450,0,0.0
618,2017,KY,289883,289883,0,0.0
...,...,...,...,...,...,...
316,2012,MA,8951,8951,0,0.0
317,2012,VA,341910,341910,0,0.0
318,2012,MS,1746,1746,0,0.0
319,2012,DE,1114,1114,0,0.0


Should get 50 states plus Puerto Rico (PR), Washington D.C. (DC), Virgin Islands (VI), and unknown (XX)

In [27]:
df['state'].unique()

array(['KS', 'VA', 'PA', 'FL', 'AK', 'SD', 'NY', 'ND', 'RI', 'ME', 'CO',
       'NM', 'XX', 'CA', 'GA', 'ID', 'NE', 'UT', 'HI', 'AR', 'AL', 'TX',
       'IL', 'NV', 'NC', 'WI', 'VT', 'MD', 'WY', 'TN', 'DC', 'OK', 'MA',
       'MN', 'DE', 'WV', 'NH', 'IA', 'OR', 'OH', 'WA', 'AZ', 'MS', 'CT',
       'MI', 'NJ', 'MT', 'IN', 'SC', 'KY', 'LA', 'MO'], dtype=object)

In [28]:
df['state'].unique().shape

(52,)

In [29]:
df[df['state'] == '']

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent


In [30]:
df[df['state'].isna()]

,year,state,state_count,src_state_count,plan_diff,plan_diff_percent
